<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_AI_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

import os

In [3]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or 'Your OPENAI API Key'

In [4]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name='gpt-3.5-turbo')

# CUDA

CUDA is a parallel computing platform and programming model created by NVIDIA. With more than 20 million downloads to date, CUDA helps developers speed up their applications by harnessing the power of GPU accelerators. 

https://blogs.nvidia.com/blog/2012/09/10/what-is-cuda-2/

# BLIP

The BLIP model was proposed in BLIP: Bootstrapping Language-Image Pre-training for Unified Vision-Language Understanding and Generation by Junnan Li, Dongxu Li, Caiming Xiong, Steven Hoi.

BLIP is a model that is able to perform various multi-modal tasks including

    Visual Question Answering
    Image-Text retrieval (Image-text matching)
    Image Captioning

https://huggingface.co/docs/transformers/model_doc/blip

# Image to Text HuggingFace Model

Model card for image captioning pretrained on COCO dataset - base architecture (with ViT large backbone).

https://huggingface.co/Salesforce/blip-image-captioning-large

In [5]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

image_to_text_model = "Salesforce/blip-image-captioning-large"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

processor = BlipProcessor.from_pretrained(image_to_text_model)
model = BlipForConditionalGeneration.from_pretrained(image_to_text_model).to(device)

In [6]:
from transformers.models.oneformer.modeling_oneformer import OneFormerModelOutput
import requests
from PIL import Image

def describeImage(image_url):
  image_object = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')
  # image
  inputs = processor(image_object, return_tensors="pt").to(device)
  outputs = model.generate(**inputs)
  return processor.decode(outputs[0], skip_special_tokens=True)

In [9]:
description = describeImage('https://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80')

In [10]:
description

'porsche 911 carros parked in a parking lot with a house in the background'

In [13]:
from langchain.tools import BaseTool

class DescribeImageTool(BaseTool):
    name = "Describe Image Tool"
    description = 'use this tool to describe an image.'

    def _run(self, url: str):
        description = describeImage(url)
        return description
    
    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")

tools = [DescribeImageTool()]

# LangChain Agent Types

https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html

## chat-conversational-react-description

a specific type of agent (chat-conversational-react-description) which expects to be used with a memory component.

https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html

In [14]:
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)

In [15]:
image_url = 'https://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80'
agent(f"Describe the following image:\n{image_url}")



> Entering new AgentExecutor chain...
{
    "action": "Describe Image Tool",
    "action_input": "https://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80"
}
Observation: porsche 911 carros parked in a parking lot with a house in the background
Thought:{
    "action": "Final Answer",
    "action_input": "The image is of a parking lot with several Porsche 911 cars parked in it. There is a house visible in the background."
}

> Finished chain.


{'input': 'Describe the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80',
 'chat_history': [],
 'output': 'The image is of a parking lot with several Porsche 911 cars parked in it. There is a house visible in the background.'}

In [16]:
agent(f"What is the brand of car in the following image:\n{image_url}")



> Entering new AgentExecutor chain...
{
    "action": "Describe Image Tool",
    "action_input": "https://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80"
}
Observation: porsche 911 carros parked in a parking lot with a house in the background
Thought:{
    "action": "Final Answer",
    "action_input": "The brand of car in the image is Porsche 911."
}

> Finished chain.


{'input': 'What is the brand of car in the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80',
 'chat_history': [HumanMessage(content='Describe the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80', additional_kwargs={}),
  AIMessage(content='The image is of a parking lot with several Porsche 911 cars parked in it. There is a house visible in the background.', additional_kwargs={})],
 'output': 'The brand of car in the image is Porsche 911.'}

In [18]:
image_url = 'https://images.unsplash.com/photo-1682228287072-5e23cbffd487?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=987&q=80'
agent(f"Please describe the following image:\n{image_url}")




> Entering new AgentExecutor chain...
```json
{
    "action": "Describe Image Tool",
    "action_input": "https://images.unsplash.com/photo-1682228287072-5e23cbffd487?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=987&q=80"
}
```
Observation: a close up of a bunch of red lanterns hanging from a ceiling
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The image is a close up of a bunch of red lanterns hanging from a ceiling."
}
```

> Finished chain.


{'input': 'Please describe the following image:\nhttps://images.unsplash.com/photo-1682228287072-5e23cbffd487?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=987&q=80',
 'chat_history': [HumanMessage(content='Describe the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80', additional_kwargs={}),
  AIMessage(content='The image is of a parking lot with several Porsche 911 cars parked in it. There is a house visible in the background.', additional_kwargs={}),
  HumanMessage(content='What is the brand of car in the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80', additional_kwargs={}),
  AIMessage(content='The brand of car in the image is Porsche 911.', additional_kwargs={}),
  HumanMessage(content

In [19]:
agent.memory.buffer

[HumanMessage(content='Describe the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80', additional_kwargs={}),
 AIMessage(content='The image is of a parking lot with several Porsche 911 cars parked in it. There is a house visible in the background.', additional_kwargs={}),
 HumanMessage(content='What is the brand of car in the following image:\nhttps://images.unsplash.com/photo-1673207520321-c27d09eb0955?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80', additional_kwargs={}),
 AIMessage(content='The brand of car in the image is Porsche 911.', additional_kwargs={}),
 HumanMessage(content='What is the brand of car in the following image:\nhttps://images.unsplash.com/photo-1679707883439-117608837612?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1035&q=80', additional_kwargs